In [61]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.chat_models import BedrockChat
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_aws import BedrockEmbeddings

prompt_template = """

Human:Use the following pieces of context to provide a 
concise answer to the question at the end. if you know the answer , give me the answer as like chat conversation. If you don't know the answer, 
just say give answer as "I don't know" in 3 word.
Read the following text inside <text></text> 
XML tags, and then answer the question:
<context>
don't consider maternity leave as type of time-off
</context>
<context>
{context}
</context>
<context>
{context}
</context>

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

def get_response_llm(llm,vectorstore_pincone,query):
    qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_pincone.as_retriever(
        search_type="similarity", search_kwargs={"k": 1}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
    answer=qa({"query":query})
    return answer

def get_claude_llm():
    llm=BedrockChat(
        credentials_profile_name='default',
        model_id='anthropic.claude-3-sonnet-20240229-v1:0',
        region_name='us-east-1',
        model_kwargs={
        "max_tokens":2048,
        "temperature": 0.1,
        "top_p": 0.9})
    return llm    

def getVectorStore():
    pc = Pinecone(api_key="312601e0-eb28-418c-82f5-f3cd8c0d14c3")
    index = pc.Index("langchain-rag-index")
    data_embeddings=BedrockEmbeddings(
    credentials_profile_name= 'default',
    model_id='amazon.titan-embed-text-v1',
    region_name='us-east-1')    
    vector_store = PineconeVectorStore(index=index, embedding=data_embeddings)
    return vector_store    

 
def lambda_handler(text, context):
    llm=get_claude_llm()
    vectorStore = getVectorStore()
    result = get_response_llm(llm,vectorStore,text)
    return result 

In [63]:
result = lambda_handler("what are different type of time off","")
result

{'query': 'what are different type of time off',
 'result': 'Based on the given context, the different types of time off mentioned are:\n\nVacation, sick leave, paid time off, holiday, and maternity leave.',
 'source_documents': [Document(metadata={'leave': 'timeoff'}, page_content='WFM enables you to configure multiple types of time off. Examples of time-off types include vacation,\nsick leave, paid time off, holiday, and maternity leave.\nBy default, WFM creates the Vacation time-off type. This time-off type cannot be deleted and does\nnot belong exclusively to any site.\nYou can configure a time-off rule for a one or multiple time-off types. When you configure multiple\ntime-off types for the same rule, the time-off balance is calculated and accrued for all time-off types\nassociated with that rule.\nUse the procedures in this topic to create, copy, edit, and delete time-off types, and associate/\ndisassociate time-off types with sites.\nYou assign time-off type/time-off rule combin